In [41]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

In [42]:
# Load the data file
file_path = './SPX_hedging.csv'
data = pd.read_csv(file_path, usecols=['S', 'Dividend', 'C_mkt', 'R', 'K', 'TTM', 'Moneyness']).dropna()
print(data.describe())

                 S   Dividend       C_mkt         R         TTM   Moneyness  \
count    85.000000  85.000000   85.000000  85.00000   85.000000   85.000000   
mean   5718.486941   1.345927  315.221176   4.80564  168.870588   18.458824   
std     201.911555   0.047218   95.263399   0.10632   35.091305  201.966260   
min    5186.330000   1.276500  111.850000   4.59072  108.000000 -514.000000   
25%    5597.120000   1.311000  257.450000   4.76095  140.000000 -103.000000   
50%    5728.800000   1.336400  311.800000   4.77997  169.000000   29.000000   
75%    5853.980000   1.372100  383.750000   4.88116  198.000000  154.000000   
max    6049.880000   1.481600  487.300000   5.01499  228.000000  350.000000   

            K  
count    85.0  
mean   5700.0  
std       0.0  
min    5700.0  
25%    5700.0  
50%    5700.0  
75%    5700.0  
max    5700.0  


In [51]:
# Black-Scholes call option pricing formula
# Removed Crank-Nicholson and replaced with Binomial Tree method for pricing
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f"{func.__name__} executed in {end_time - start_time:.6f} seconds")
        return result
    return wrapper

def binomial_tree_call_price(S, K, T, r, sigma, N=100):
    dt = T / N  # Time step size
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Initialize asset prices at maturity
    asset_prices = np.zeros(N + 1)
    for i in range(N + 1):
        asset_prices[i] = S * (u ** (N - i)) * (d ** i)

    # Initialize option values at maturity
    option_values = np.maximum(asset_prices - K, 0)

    # Backward induction to calculate option value at t=0
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])

    return option_values[0]

# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        return binomial_tree_call_price(S, K, T, r, sigma) - C_mkt
    try:
        return bisect(objective, 1e-6, 5)  # Searching for sigma in a reasonable range
    except ValueError:
        return np.nan


In [57]:
@timer
def delta_hedge(data):
    # Initialize strategy variables
    position = 0  # Current asset position
    cash = 0      # Cash balance
    portfolio_values = []  # Store portfolio values over time
    predicted_prices = []  # Store predicted option prices

    # Use a dynamic window of 5 rows for prediction and hedging
    window_size = 5
    for start in range(0, len(data), window_size):
        window_data = data[start:start + window_size]
        if len(window_data) < window_size:
            break  # Skip incomplete windows
    
        for i, row in window_data.iterrows():
            S = row['S']  # Current stock price
            K = row['K']  # Calculate strike price based on Moneyness
            T = row['TTM'] / 252  # Convert time to expiration to years
            r = row['R'] / 100  # Convert interest rate to decimal
            C_mkt = row['C_mkt']  # Market option price
    
            # Calculate implied volatility
            sigma = implied_volatility(C_mkt, S, K, T, r)
    
            # Skip this row if implied volatility could not be calculated
            if np.isnan(sigma):
                continue
    
            # Calculate Delta using Binomial Tree method
            epsilon = 1e-4  # Small change for Delta calculation
            V_up = binomial_tree_call_price(S * (1 + epsilon), K, T, r, sigma)
            V_down = binomial_tree_call_price(S * (1 - epsilon), K, T, r, sigma)
            delta = (V_up - V_down) / (2 * S * epsilon)
    
            # Calculate predicted option price using Binomial Tree method
            predicted_price = binomial_tree_call_price(S, K, T, r, sigma)
            predicted_prices.append(predicted_price)
    
            # Determine the target position based on Delta
            target_position = -delta
            position_change = target_position - position
    
            # Update cash and asset position
            cash -= position_change * S
            position = target_position
    
            # Record portfolio value
            portfolio_values.append(position * S + cash)
    return portfolio_values, predicted_prices

In [58]:
portfolio_values, predicted_prices = delta_hedge(data)
# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()

print(predicted_prices_df.describe())
print('-------------------')
print("market data")
print(data['C_mkt'].describe())
print('-------------------')
print(predicted_prices_df.head())
print('------------------')
returns = portfolio_values.loc[
    (pd.notnull(portfolio_values['Returns'])) & (np.isfinite(portfolio_values['Returns'])),
    'Returns'
]

print(returns.describe())

volatility = returns.std()  # Calculate return volatility
cumulative_returns = (1 + returns).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()

# Output risk metrics
print("return volatility:", volatility, "max_drawdown", max_drawdown)

/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_38218/3442528316.py:31: RuntimeWarning: overflow encountered in scalar multiply
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])
/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_38218/3442528316.py:31: RuntimeWarning: invalid value encountered in scalar add
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])
/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_38218/3442528316.py:31: RuntimeWarning: overflow encountered in scalar add
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])


delta_hedge executed in 8.140725 seconds
       Predicted Price
count        50.000000
mean        275.570000
std          97.314639
min         111.850000
25%         215.887500
50%         259.725000
75%         311.425000
max         483.900000
-------------------
market data
count     85.000000
mean     315.221176
std       95.263399
min      111.850000
25%      257.450000
50%      311.800000
75%      383.750000
max      487.300000
Name: C_mkt, dtype: float64
-------------------
   Predicted Price
0           147.20
1           125.90
2           111.85
3           153.75
4           145.55
------------------
count    48.000000
mean      0.169464
std       0.823398
min      -0.655373
25%      -0.034983
50%       0.037451
75%       0.184505
max       5.542917
Name: Returns, dtype: float64
return volatility: 0.8233976481210112 max_drawdown 7.079775104271683
